---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [130]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [138]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
home = pd.read_csv('City_Zhvi_AllHomes.csv')
utown = pd.read_table('university_towns.txt',header=-1)
gdp = pd.read_excel('gdplev.xls',skiprows=4)
gdp = gdp.ix[gdp[gdp.ix[:,4] == '2000q1'].index.tolist()[0]:,[4,6]]
gdp.reset_index(drop=True,inplace=True)
gdp.columns = ['quater','gdp']
gdp.head()
# len(gdp)
# utown.head()
# len(utown)
# type(utown)

,quater,gdp
0,2000q1,12359.1
1,2000q2,12592.5
2,2000q3,12607.7
3,2000q4,12679.3
4,2001q1,12643.3


In [132]:
def get_list_of_university_towns():
    
    st = utown[utown.iloc[:,0].str.contains('\[edit\]')]
    st = st.reset_index()
    # # st['index']
    st[0] = st[0].apply(lambda x: x.split('[')[0])
    # # st = st.loc[np.repeat(st.index.values,st['index'])]

    temp = utown.drop(utown.index[st['index']])
    temp = temp.reset_index(drop=True)
    temp2 = st['index'].diff()[1:]
    temp2 = (temp2.append(pd.Series(len(utown)-st['index'].iloc[-1]))-1).astype('int')
    # # .append(len(utown)-st['index'])
    temp2.reset_index(drop=True,inplace=True)
    temp2 = st.loc[np.repeat(st.index.values,temp2)]
    temp2.reset_index(drop=True,inplace=True)
    temp2.drop('index',axis=1,inplace=True)
    answer = pd.DataFrame(temp2[0], columns=['State'])
    answer['RegionName'] = temp
    answer['RegionName'] = answer['RegionName'].apply(lambda x: x.split('(')[0].rstrip())
    # answer
    return answer

# get_list_of_university_towns()
    



In [140]:
def get_recession_start():
    # answer = []
    now = False
    for i in range(1,len(gdp)-1):

        if((gdp.gdp[i]<gdp.gdp[i-1]) & (gdp.gdp[i]>gdp.gdp[i+1]) & ~now):
            answer = gdp.quater[i]
            now = True
        if((gdp.gdp[i]>gdp.gdp[i-1]) & (gdp.gdp[i]<gdp.gdp[i+1]) & now):
            now =False
            
    return answer

get_recession_start()

'2008q3'

In [141]:
def get_recession_end():
    # answer = []
    now = False
    for i in range(0,len(gdp)-2):

        if((gdp.gdp[i]>gdp.gdp[i+1]) & (gdp.gdp[i+1]>gdp.gdp[i+2]) & ~now):
            now = True
        if((gdp.gdp[i]<gdp.gdp[i+1]) & (gdp.gdp[i+1]<gdp.gdp[i+2]) & now):
            answer = gdp.quater[i+2]
            now = False
            
    return answer

get_recession_end()

'2009q4'

In [127]:
def get_recession_bottom():
    start = get_recession_start()
    end = get_recession_end()
    # bottom = gdp.quater[gdp.]

    answer = gdp.quater.iloc[gdp.gdp[(gdp.quater>=start) & (gdp.quater<=end)].idxmin()]
    return answer

# get_recession_bottom()

In [128]:
def convert_housing_data_to_quarters():
    # convert_housing_data_to_quarters()
#     def quarter_mean(x):
#         num_quarter = int(len(x)/3.0)+1
#         answer = pd.Series(np.zeros(num_quarter))
#         for i in range(0, num_quarter):
#             if(i<num_quarter-1):
#                 answer[i] = np.mean(x[(i*3):((i+1)*3)])
#             else:
#                 answer[i] = np.mean(x[(i*3):])
#         return answer
#     temp = home.iloc[:,-200:]
#     temp['State'] = home['State'].map(states)
#     temp['RegionName'] = home['RegionName']
#     # home.iloc[1]
#     temp.set_index(['State','RegionName'],drop=True,inplace=True)
#     temp.sort_index(inplace=True)

#     answer = temp.apply(quarter_mean, axis=1)

    
    temp = home.iloc[:,-200:]
    temp['State'] = home['State'].map(states)
    temp['RegionName'] = home['RegionName']
    # home.iloc[1]
    temp.set_index(['State','RegionName'],drop=True,inplace=True)
    temp.sort_index(inplace=True)
    answer = temp.groupby(np.arange(temp.shape[1])//3, axis=1).apply(np.mean,1)
    answer.rename(columns=lambda x: str(2000+int(x/4.0))+'q'+str(x%4+1),inplace=True)

    return answer

# convert_housing_data_to_quarters()

# temp.head()
# convert_housing_data_to_quarters()
# # convert_housing_data_to_quarters()
# def quarter_mean(x):
#     num_quarter = int(len(x)/3.0)+1
#     answer = pd.Series(np.zeros(num_quarter))
#     for i in range(0, num_quarter):
#         if(i<num_quarter-1):
#             answer[i] = np.mean(x[(i*3):((i+1)*3)])
#         else:
#             answer[i] = np.mean(x[(i*3):])
#     return answer
# temp = home.iloc[:,-200:]
# temp['State'] = home['State'].map(states)
# temp['RegionName'] = home['RegionName']
# # home.iloc[1]
# temp.set_index(['State','RegionName'],drop=True,inplace=True)
# temp.sort_index(inplace=True)

# answer = temp.apply(quarter_mean, axis=1)

# type(answer)

# # quarter_mean(temp.iloc[0])

In [129]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start = get_recession_start()
    bottom = get_recession_end()
    home_price = convert_housing_data_to_quarters()
    idx = [home_price.columns.get_loc(start)-1, home_price.columns.get_loc(bottom)]
    home_price2 = home_price.iloc[:,idx]
    home_price2['price_ratio'] = home_price2.iloc[:,0]/home_price2.iloc[:,1]

    # start
    # bottom
    # home_price2
    utown_list = get_list_of_university_towns()
    utown_price = pd.merge(home_price2, utown_list, how='inner', left_index=True, right_on=['State', 'RegionName'])
    utown_price.reset_index(drop=True, inplace=True)
    utown_price = utown_price['price_ratio']
    utown_ratio = utown_price.dropna()

    temp = pd.merge(home_price2, utown_list, how='left', left_index=True, right_on=['State', 'RegionName'], indicator=True)
    non_utown_price = temp[temp['_merge']=='left_only']
    non_utown_price.sort(['State','RegionName'],inplace=True)
    non_utown_price.reset_index(drop=True, inplace=True)
    non_utown_price = non_utown_price['price_ratio']
    non_utown_ratio = non_utown_price.dropna()

    # type(utown_price)
    # type(non_utown_price)

    # print(np.mean(utown_price))
    # print(np.mean(non_utown_price))
    # print(np.std(utown_price))
    # print(np.std(non_utown_price))


    test = ttest_ind(utown_ratio, non_utown_ratio)
    if(test[1]<0.01):
        different = True
        better = 'university town'
    else:
        different = False
        better = 'non-university town'
        
    answer = (different, test[1], better)
    # utown_list
    # len(utown_price)
    # utown_list
    # home[home['RegionName']=='Auburn']

    # utown_list
    # home_price2
    # utown_price_ratio = home_price2[home_price2.index.isin(utown_list)]
    # home_price2[home_price2.index == {'Alabama','Auburn'}]

    # utown_price_ratio
    # utown_list
    # home_price2
    # utown_price_ratio = home_price2

    # home_price.columns
    # idx
    
    return answer

# run_ttest()